<H1>KAUSTUBH RAYKAR <BR> <H2> PRN : 21070126048

In [103]:
import json
import pandas as pd

with open('/content/drive/MyDrive/DataEngineeringQ2.json') as file:
    data = json.load(file)

parsed_data = []

for record in data:
    patient = record['patientDetails']
    medicines = record['consultationData']['medicines']

    for medicine in medicines:
        parsed_data.append({
            'patient_id': patient['_id'],
            'first_name': patient.get('firstName', ''),
            'last_name': patient.get('lastName', ''),
            'email': patient.get('emailId', ''),
            'gender': patient.get('gender', ''),
            'birth_date': patient.get('birthDate', ''),
            'phone_number': record.get('phoneNumber', ''),
            'medicine_id': medicine['medicineId'],
            'medicine_name': medicine['medicineName'],
            'frequency': medicine['frequency'],
            'duration': medicine['duration'],
            'duration_in': medicine['durationIn'],
            'instruction': medicine['instruction'],
            'is_active': medicine['isActive']
        })

df = pd.DataFrame(parsed_data)

df.head()


,patient_id,first_name,last_name,email,gender,birth_date,phone_number,medicine_id,medicine_name,frequency,duration,duration_in,instruction,is_active
0,T6hb630b3,Css,,,,None,96686896670,619404,A,1-0-1,90,DAYS,AFTER MEAL,True
1,T6hb630b3,Css,,,,None,96686896670,619804,B,1-0-1,90,DAYS,AFTER MEAL,False
2,T6h33b300,Lokesh,,,M,1996-05-16T18:30:00.000Z,9496368916,619404,A,1-0-1,90,DAYS,AFTER MEAL,True
3,T6h33b300,Lokesh,,,M,1996-05-16T18:30:00.000Z,9496368916,619804,C,1-0-1,30,DAYS,AFTER MEAL,False
4,TjhB4373,Shila,Das,,,,7787204833,619404,B,1-0-1,10,DAYS,AFTER MEAL,True


In [104]:
df['birth_date'] = pd.to_datetime(df['birth_date'], errors='coerce')

df['age'] = df['birth_date'].apply(lambda x: pd.Timestamp.now().year - x.year if pd.notnull(x) else None)

medicine_agg = df.groupby(['patient_id', 'first_name', 'last_name']).agg(
    total_medicines=('medicine_id', 'count'),
    active_medicines=('is_active', 'sum'),
    inactive_medicines=('is_active', lambda x: (~x).sum())
).reset_index()

medicine_agg.head()

,patient_id,first_name,last_name,total_medicines,active_medicines,inactive_medicines
0,04Re22d,Tejpal,,2,2,0
1,6df4R5b,Bhavika,Ben Panchal,1,1,0
2,Pdh97S6,Jay,Sharma,7,4,3
3,R53Et5W,Rajeshwari,,4,2,2
4,T4E7Ge6,Nivitha,,3,3,0


In [105]:
# Check for missing values in essential columns
missing_values = df[['patient_id', 'medicine_id', 'medicine_name']].isnull().sum()

unique_duration_units = df['duration_in'].unique()

missing_values, unique_duration_units

(patient_id       0
 medicine_id      0
 medicine_name    0
 dtype: int64,
 array(['DAYS'], dtype=object))

In [106]:
# Share percentage of missing values, for the columns firstName, lastName and DOB from patientDetails.
total_records = len(df)
missing_first_name = df['first_name'].eq('').sum() / total_records * 100
missing_last_name = df['last_name'].eq('').sum() / total_records * 100
missing_birth_date = df['birth_date'].isnull().sum() / total_records * 100

missing_values_percentage = f"{missing_first_name:.2f}, {missing_last_name:.2f}, {missing_birth_date:.2f}"
missing_values_percentage


'0.00, 66.67, 31.82'

In [107]:
# What is the percentage of female gender after imputation of gender column basis mode.
mode_gender = df['gender'].mode()[0]
df['gender'].replace('', mode_gender, inplace=True)

female_percentage = (df['gender'] == 'F').sum() / len(df) * 100
female_percentage_rounded = f"{female_percentage:.2f}"
female_percentage_rounded


'65.15'

In [115]:
#What is count of Adult?

df['age'] = df['birth_date'].apply(lambda x: pd.Timestamp.now().year - x.year if pd.notnull(x) else None)

print(df['age'].head())

def categorize_age(age):
    if age is None:
        return 'Unknown'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'

df['ageGroup'] = df['age'].apply(categorize_age)

adult_count = df['ageGroup'].value_counts().get('Adult', 0)
adult_count

0     NaN
1     NaN
2    28.0
3    28.0
4     NaN
Name: age, dtype: float64


45

In [109]:
# Calculate average number of medicines prescribed
average_medicines = df.groupby('patient_id').size().mean()
average_medicines_rounded = f"{average_medicines:.2f}"
average_medicines_rounded


'2.75'

In [110]:
#Determine the 3rd most frequently prescribed medicineName

most_prescribed = df['medicine_name'].value_counts().head(10)

frequency_distribution = df['frequency'].value_counts()

most_prescribed, frequency_distribution


(medicine_name
 A    17
 D    17
 C    13
 B    12
 E     6
 G     1
 Name: count, dtype: int64,
 frequency
 1-0-1    65
 1-1-1     1
 Name: count, dtype: int64)

In [111]:
# Find the percentage distribution of total active and inactive medicines.

total_medicines = len(df)
active_medicines_percentage = df['is_active'].sum() / total_medicines * 100
inactive_medicines_percentage = (total_medicines - df['is_active'].sum()) / total_medicines * 100

distribution_percentage = f"{active_medicines_percentage:.2f}, {inactive_medicines_percentage:.2f}"
distribution_percentage


'69.70, 30.30'

In [112]:
#Design a logic to check whether a phone number is valid or not.

def is_valid_mobile(number):
    if number.startswith('+91'):
        number = number[3:]
    elif number.startswith('91'):
        number = number[2:]

    if number.isdigit() and len(number) == 10 and 6000000000 <= int(number) <= 9999999999:
        return True
    else:
        return False

df['isValidMobile'] = df['phone_number'].apply(is_valid_mobile)

valid_phone_numbers_count = df['isValidMobile'].sum()

(valid_phone_numbers_count)

51

In [113]:
# What is the pearson correlation between the number of prescribed medicines and the patient's age?

medicines_per_patient = df.groupby('patient_id').size()

df_medicines_count = df.drop_duplicates(subset='patient_id').set_index('patient_id')
df_medicines_count['num_medicines'] = medicines_per_patient


pearson_correlation = df_medicines_count[['num_medicines', 'age']].corr().iloc[0, 1]
pearson_correlation_rounded = f"{pearson_correlation:.2f}"

(pearson_correlation_rounded)

'-0.13'